In [2]:
import pandas as pd

df = pd.read_csv("/Users/viktoriiaovsianik/Documents/Uni/04_WS2024/02_Realtime Viz/02_Project/Project/archive/flights_sample_3m.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 32 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   FL_DATE                  object 
 1   AIRLINE                  object 
 2   AIRLINE_DOT              object 
 3   AIRLINE_CODE             object 
 4   DOT_CODE                 int64  
 5   FL_NUMBER                int64  
 6   ORIGIN                   object 
 7   ORIGIN_CITY              object 
 8   DEST                     object 
 9   DEST_CITY                object 
 10  CRS_DEP_TIME             int64  
 11  DEP_TIME                 float64
 12  DEP_DELAY                float64
 13  TAXI_OUT                 float64
 14  WHEELS_OFF               float64
 15  WHEELS_ON                float64
 16  TAXI_IN                  float64
 17  CRS_ARR_TIME             int64  
 18  ARR_TIME                 float64
 19  ARR_DELAY                float64
 20  CANCELLED                float64
 21  CANCELLA

In [7]:
df["ARR_TIME"].min()

1.0

In [9]:
df_trunc = df[:10000]

In [16]:
df_trunc.to_excel("test.xlsx", index=False)

In [25]:
import pandas as pd

# Ensure FL_DATE is in datetime format
df['FL_DATE'] = pd.to_datetime(df['FL_DATE'], errors='coerce')

# Extract the year from the FL_DATE column
df['Year'] = df['FL_DATE'].dt.year
# Mark flights as delayed if ARR_DELAY >= 15
df['DELAYED'] = df['ARR_DELAY'] >= 15

agg_df = df.groupby(['ORIGIN', 'ORIGIN_CITY', 'AIRLINE', 'Year']).agg(
    flights=('FL_NUMBER', 'count'),  # Count the number of flights
    delayed=('DELAYED', 'sum'),  # Sum the delayed flights (True values)
    # Calculate the average delay only for delayed flights
    avg_delay=('ARR_DELAY', lambda x: x[x >= 15].mean() if len(x[x >= 15]) > 0 else None), 
    connections=('DEST', lambda x: list(set(x))),  # List of unique destination airports
    pct_delayed=('DELAYED', 'mean')  # Percentage of delayed flights
).reset_index()

In [26]:
agg_df

,ORIGIN,ORIGIN_CITY,AIRLINE,Year,flights,delayed,avg_delay,connections,pct_delayed
0,ABE,"Allentown/Bethlehem/Easton, PA",Allegiant Air,2019,111,23,115.913043,"[FLL, SFB, BNA, PIE, MYR, SAV, PGD]",0.207207
1,ABE,"Allentown/Bethlehem/Easton, PA",Allegiant Air,2020,117,24,137.625000,"[FLL, SFB, SRQ, BNA, MDW, PIE, PGD, SAV, MYR]",0.205128
2,ABE,"Allentown/Bethlehem/Easton, PA",Allegiant Air,2021,131,24,63.625000,"[FLL, SFB, SRQ, BNA, PIE, PGD, SAV, MYR]",0.183206
3,ABE,"Allentown/Bethlehem/Easton, PA",Allegiant Air,2022,139,30,63.100000,"[FLL, SFB, SRQ, BNA, PIE, PGD, MYR]",0.215827
4,ABE,"Allentown/Bethlehem/Easton, PA",Allegiant Air,2023,81,23,74.130435,"[FLL, SFB, SRQ, BNA, DEN, PIE, MYR, PGD]",0.283951
...,...,...,...,...,...,...,...,...,...
9378,YUM,"Yuma, AZ",SkyWest Airlines Inc.,2019,186,14,101.857143,"[PHX, DFW]",0.075269
9379,YUM,"Yuma, AZ",SkyWest Airlines Inc.,2020,112,10,121.100000,"[PHX, DFW]",0.089286
9380,YUM,"Yuma, AZ",SkyWest Airlines Inc.,2021,184,17,177.882353,"[PHX, DFW]",0.092391
9381,YUM,"Yuma, AZ",SkyWest Airlines Inc.,2022,127,23,57.521739,"[PHX, DFW]",0.181102


In [23]:
agg_df[agg_df["ORIGIN"]=="ABQ"]

,ORIGIN,ORIGIN_CITY,AIRLINE,Year,flights,delayed,avg_delay,connections,pct_delayed
35,ABQ,"Albuquerque, NM",Alaska Airlines Inc.,2019,54,11,2.907407,[SEA],0.203704
36,ABQ,"Albuquerque, NM",Alaska Airlines Inc.,2020,43,3,-8.186047,"[SAT, PDX, SEA]",0.069767
37,ABQ,"Albuquerque, NM",Alaska Airlines Inc.,2021,38,3,-6.810811,[SEA],0.078947
38,ABQ,"Albuquerque, NM",Alaska Airlines Inc.,2022,52,10,-0.653846,"[PDX, SEA]",0.192308
39,ABQ,"Albuquerque, NM",Alaska Airlines Inc.,2023,45,11,7.139535,"[PDX, SEA]",0.244444
...,...,...,...,...,...,...,...,...,...
92,ABQ,"Albuquerque, NM",United Air Lines Inc.,2019,91,18,9.079545,"[DEN, ORD, SFO, IAH]",0.197802
93,ABQ,"Albuquerque, NM",United Air Lines Inc.,2020,29,4,-4.142857,"[DEN, ORD, IAH]",0.137931
94,ABQ,"Albuquerque, NM",United Air Lines Inc.,2021,36,4,-0.027778,"[DEN, ORD, IAH]",0.111111
95,ABQ,"Albuquerque, NM",United Air Lines Inc.,2022,164,24,6.777778,"[DEN, ORD, SFO, IAH]",0.146341


In [ ]:
ORIGIN,ORIGIN_CITY,AIRLINE,Year,flights,delayed,avg_delay,connections,pct_delayed,STATE,LATITUDE,LONGITUDE

In [ ]:


# Merge with additional information from the dfa DataFrame (assuming you have a `dfa` DataFrame)
agg_df = agg_df.merge(dfa[['ORIGIN', 'STATE', 'LATITUDE', 'LONGITUDE']], on='ORIGIN', how='left')

# Convert pct_delayed to percentage (multiply by 100)
agg_df['pct_delayed'] = agg_df['pct_delayed'] * 100

# Display the result
print(agg_df.head())

